# 2D and 3D Coordinate Transformations

This course covers the fundamentals of coordinate transformations in 2D and 3D spaces. We'll start with simple 2D rotations, then extend to translations, and finally work with homogeneous coordinates to represent both in a unified way. Later, we'll move to 3D transformations and apply these concepts to computer vision with a pinhole camera model.

**Topics covered:**
- 2D coordinate frames and rotations
- Translations and their representation
- Homogeneous coordinates
- 3D transformations
- Pinhole camera model and projections

In [1]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from math import cos, sin, radians

# Import custom visualization libraries
from lib.visualization2d import create_2d_visualization, create_interactive_2d_visualization, create_dual_point_interactive_visualization, point_in_frame

## 1. 2D Coordinate Frames and Rotations

### 1.1 Coordinate Frames in 2D

A coordinate frame in 2D consists of two perpendicular axes (X and Y) and an origin point. Any point in the 2D space can be represented by its coordinates (x, y) with respect to a particular frame.

Let's start with a simple example: a point at coordinates (1, 1) in the standard world frame.

In [2]:
# Create a visualization of a point in the world frame
point_coords = [1, 1]  # (x, y) coordinates

# Define world frame (at origin, no rotation)
world_frame = [0, 0, 0, ""]  # [x, y, rotation_deg, name]

# Define the point
point = [point_coords[0], point_coords[1], "P", "blue", 0]  # [x, y, label, color, frame_index]

# Create and display the visualization
fig = create_2d_visualization(
    points=[point],
    frames=[world_frame],
    show_coords=True,
    show_labels=True
)

fig.show()

### 1.2 Introducing a Rotated Frame

Now, let's introduce a second frame that is rotated 30 degrees clockwise around the origin.

When we have different reference frames, the same point in space will have different coordinate values in each frame. We need to understand how to transform coordinates from one frame to another.

In [3]:
# Create a visualization with two frames
# World frame
world_frame = [0, 0, 0, ""]  # [x, y, rotation_deg, name]

# Rotated frame (30 degrees clockwise, represented as -30 degrees)
rotation_angle = -30  # degrees (negative means clockwise)
rotated_frame = [0, 0, rotation_angle, "R"]  # [x, y, rotation_deg, name]

# Define the point
point = [1, 1, "P", "blue", 0]  # [x, y, label, color, frame_index]

# Create and display the visualization
fig = create_2d_visualization(
    points=[point],
    frames=[world_frame, rotated_frame],
    show_coords=True,
    show_labels=True
)

fig.show()

### 1.3 Coordinate Transformation: Mathematics Behind Rotation

Let's figure out how to find the coordinates of our point P(1, 1) in the rotated frame.

#### Simple explanation of coordinate transformation

Imagine you're standing on a grid, looking in a certain direction (let's call it 'forward'). You see a point P at position (1, 1), meaning it's 1 step right and 1 step forward from you.

Now, if you turn 30 degrees to your right (clockwise), the point P hasn't moved, but from your new perspective, its coordinates will change. This is exactly what happens with coordinate frames!

#### How to calculate the new coordinates

When we rotate a frame clockwise by an angle θ, we can find the new coordinates using this special pattern:

$$\begin{bmatrix} x' \\ y' \end{bmatrix} = \begin{bmatrix} \cos(\theta) & -\sin(\theta) \\ \sin(\theta) & \cos(\theta) \end{bmatrix} \begin{bmatrix} x \\ y \end{bmatrix}$$

This looks complicated, but it just means:
- x' = x·cos(θ) - y·sin(θ)
- y' = x·sin(θ) + y·cos(θ)
<!-- 
#### Why this works (a visual explanation)

Let's break this down in a way that makes sense without advanced math:

1. The original coordinate system has two directions: right (x-axis) and up (y-axis)

2. When we rotate the frame by θ degrees clockwise:
   - The new 'right' direction (x'-axis) is rotated clockwise from the original right
   - The new 'up' direction (y'-axis) is rotated clockwise from the original up

3. To find how much of a point is in the new 'right' direction (x'):
   - We take the amount it was in the original right direction (x) but scaled by how much the new right aligns with the old right (cos(θ))
   - Minus the amount it was in the original up direction (y) but scaled by how much the new right aligns with the old up (-sin(θ))

4. Similarly for the new 'up' direction (y'):
   - We take the amount it was in the original right direction (x) but scaled by how much the new up aligns with the old right (sin(θ))
   - Plus the amount it was in the original up direction (y) but scaled by how much the new up aligns with the old up (cos(θ))

This rotation matrix for clockwise rotation is:

$$\begin{bmatrix} \cos(\theta) & -\sin(\theta) \\ \sin(\theta) & \cos(\theta) \end{bmatrix}$$ -->

Notice how it differs from the counterclockwise rotation matrix by the sign of the sine terms.

Let's calculate what happens to our point (1, 1) when viewed from a frame rotated by 30 degrees clockwise:

In [4]:
# Calculate coordinates in rotated frame
theta = radians(-rotation_angle)  # Convert to positive radians for clockwise rotation
x, y = 1, 1  # Original point coordinates

# Let's write out the formula step by step
x_prime = x * cos(theta) - y * sin(theta)
y_prime = x * sin(theta) + y * cos(theta)

print(f"Angle of rotation: {-rotation_angle} degrees clockwise")
print(f"Using the clockwise rotation formulas:")
print(f"x' = x·cos({-rotation_angle}°) - y·sin({-rotation_angle}°) = {x}·{cos(theta):.4f} - {y}·{sin(theta):.4f} = {x_prime:.4f}")
print(f"y' = x·sin({-rotation_angle}°) + y·cos({-rotation_angle}°) = {x}·{sin(theta):.4f} + {y}·{cos(theta):.4f} = {y_prime:.4f}")

# Let's do the same calculation using the rotation matrix
R = np.array([
    [cos(theta), -sin(theta)],
    [sin(theta), cos(theta)]
])

point_in_world = np.array([x, y])
point_in_rotated = R @ point_in_world

print("\nUsing the rotation matrix (same result, just written differently):")
print(f"[ {R[0,0]:.4f}  {R[0,1]:.4f} ] × [ {x} ] = [ {point_in_rotated[0]:.4f} ]")
print(f"[ {R[1,0]:.4f}  {R[1,1]:.4f} ]   [ {y} ]   [ {point_in_rotated[1]:.4f} ]")

print("\nSummary:")
print(f"Point coordinates in world frame: ({x}, {y})")
print(f"Point coordinates in rotated frame: ({point_in_rotated[0]:.4f}, {point_in_rotated[1]:.4f})")

# Visualize this transformation
fig = create_2d_visualization(
    points=[[x, y, "P", "blue", 1]],
    frames=[[0, 0, 0, "World"], [0, 0, rotation_angle, "Rotated"]],
    show_coords=True,
    show_labels=True
)
fig.show()

Angle of rotation: 30 degrees clockwise
Using the clockwise rotation formulas:
x' = x·cos(30°) - y·sin(30°) = 1·0.8660 - 1·0.5000 = 0.3660
y' = x·sin(30°) + y·cos(30°) = 1·0.5000 + 1·0.8660 = 1.3660

Using the rotation matrix (same result, just written differently):
[ 0.8660  -0.5000 ] × [ 1 ] = [ 0.3660 ]
[ 0.5000  0.8660 ]   [ 1 ]   [ 1.3660 ]

Summary:
Point coordinates in world frame: (1, 1)
Point coordinates in rotated frame: (0.3660, 1.3660)


### 1.4 Interactive Exploration of Frame Rotations

To better understand the concept of coordinate transformations between frames, let's use an interactive visualization. The following tool allows you to:

1. Adjust the position and rotation of a second reference frame
2. Move a point around in the 2D space
3. See the point's coordinates in both the world frame and the rotated frame

Try changing the sliders to see how the coordinates of the point change in different reference frames:

In [ ]:
# Create an interactive visualization with one point
controls, output = create_interactive_2d_visualization()

Output()

## 2. Translations in 2D

So far, we've looked at how to transform coordinates when the reference frame is rotated. Now, let's explore what happens when a reference frame is translated (moved) to a new origin.

### 2.1 Understanding Translations

Translation is simply a shift in position. While rotation is about changing the orientation of the axes, translation is about changing the origin of the coordinate system.

When a frame is translated, the transformation of coordinates is straightforward—we just need to add or subtract the offset between the origins of the frames:

In [6]:
# Create a visualization of a translated frame

# World frame at origin
world_frame = [0, 0, 0, "World"] 

# Translated frame (moved to position (1, 0.5) with no rotation)
translated_frame = [1.0, 0.5, 0, "T"]

# Define a point
point_coords = [1.5, 1.0]
point = [point_coords[0], point_coords[1], "P", "blue", 1]

# Create and display the visualization
fig = create_2d_visualization(
    points=[point],
    frames=[world_frame, translated_frame],
    show_coords=True,
    show_labels=True
)

fig.show()

### 2.2 Translation Mathematics

The mathematics of translation is much simpler than rotation. To find the coordinates of a point in a translated frame, we simply subtract the origin of the new frame from the point's world coordinates:

$$\begin{bmatrix} x' \\ y' \end{bmatrix} = \begin{bmatrix} x \\ y \end{bmatrix} - \begin{bmatrix} t_x \\ t_y \end{bmatrix}$$

Where:
- $(x, y)$ are the coordinates in the world frame
- $(x', y')$ are the coordinates in the translated frame
- $(t_x, t_y)$ is the translation vector (the position of the new frame's origin in the world frame)

Let's see this in practice with our point P:

In [7]:
# Calculate coordinates in the translated frame
x, y = point_coords
tx, ty = translated_frame[0], translated_frame[1]

# Translation formula
x_prime = x - tx
y_prime = y - ty

print(f"Point coordinates in world frame: ({x}, {y})")
print(f"Translation vector: ({tx}, {ty})")
print(f"Point coordinates in translated frame: ({x_prime}, {y_prime})")

# Verify with our library function
frames = [world_frame, translated_frame]
rx, ry = point_in_frame(x, y, tx, ty, 0)  # Last parameter is rotation (0 in this case)
print(f"Verification using point_in_frame function: ({rx}, {ry})")

# Visualize with both coordinate representations
point_world = [x, y, "", "blue", 0]
point_translated = [x, y, "P ", "red", 1]

fig = create_2d_visualization(
    points=[point_world, point_translated],
    frames=frames,
    show_coords=True,
    show_labels=True
)

fig.show()

Point coordinates in world frame: (1.5, 1.0)
Translation vector: (1.0, 0.5)
Point coordinates in translated frame: (0.5, 0.5)
Verification using point_in_frame function: (0.5, 0.5)


### 2.3 Combined Rotation and Translation

In real-world applications, reference frames are often both rotated and translated relative to the world frame. To transform coordinates in this case, we need to combine both operations.

The order of operations matters:
1. First, translate the point to account for the difference in origins
2. Then, rotate the point around the new origin

Let's explore this interactively to better understand how the combination of translation and rotation affects coordinates:

In [8]:
# Create an interactive visualization to explore combined translation and rotation
controls, output = create_interactive_2d_visualization()

Output()

This interactive tool allows you to:
1. Change the position of Frame 1 (translation) using the X and Y sliders
2. Change the rotation of Frame 1 using the Rotation slider
3. Move the point P around and see its coordinates in both frames

Try the following experiments:
- Set Frame 1's rotation to 0 and change only its position to see pure translation effects
- Keep Frame 1 at the origin and change only its rotation to see pure rotation effects
- Combine both translation and rotation to see how they work together

Let's now analyze the mathematics behind this transformation:

In [9]:
# Define parameters for a combined transformation
point_coords = [1.5, 1.0]  # Point in world frame
tx, ty = 1.0, 0.5         # Translation vector
rotation = -30             # Rotation in degrees (negative for clockwise)
theta = radians(-rotation) # Convert to positive radians for clockwise rotation

print(f"Point in world frame: ({point_coords[0]}, {point_coords[1]})")
print(f"Frame translation: ({tx}, {ty})")
print(f"Frame rotation: {rotation} degrees clockwise")

# Step 1: Calculate translation effect
x_translated = point_coords[0] - tx
y_translated = point_coords[1] - ty

print(f"\nStep 1 - After translation:")
print(f"New coordinates: ({x_translated}, {y_translated})")

# Step 2: Calculate rotation effect (using clockwise rotation matrix)
R = np.array([
    [cos(theta), -sin(theta)],
    [sin(theta), cos(theta)]
])

# Apply rotation to translated point
point_translated = np.array([x_translated, y_translated])
point_transformed = R @ point_translated

print(f"\nStep 2 - After rotation:")
print(f"Rotation matrix:\n{R}")
print(f"Final coordinates in transformed frame: ({point_transformed[0]:.4f}, {point_transformed[1]:.4f})")

# Visualize the result
# World frame
world_frame = [0, 0, 0, "World"] 

# Transformed frame
transformed_frame = [tx, ty, rotation, "TR"]

# Define points
point_world = [point_coords[0], point_coords[1], "", "blue", 0]
point_transformed_vis = [point_coords[0], point_coords[1], "P", "red", 1]

# Create and display the visualization
fig = create_2d_visualization(
    points=[point_world, point_transformed_vis],
    frames=[world_frame, transformed_frame],
    show_coords=True,
    show_labels=True
)

fig.show()

Point in world frame: (1.5, 1.0)
Frame translation: (1.0, 0.5)
Frame rotation: -30 degrees clockwise

Step 1 - After translation:
New coordinates: (0.5, 0.5)

Step 2 - After rotation:
Rotation matrix:
[[ 0.8660254 -0.5      ]
 [ 0.5        0.8660254]]
Final coordinates in transformed frame: (0.1830, 0.6830)


The key insight here is that coordinate transformations between frames that are both translated and rotated require two separate operations applied in the correct order.

### 2.4 The Problem: Two Different Operations

Let's recap what we've learned about coordinate transformations:

1. For **rotation**, we use a matrix multiplication:
   - x' = x·cos(θ) - y·sin(θ)
   - y' = x·sin(θ) + y·cos(θ)
   
   Which we can write as:
   
   $$\begin{bmatrix} x' \\ y' \end{bmatrix} = \begin{bmatrix} \cos(\theta) & -\sin(\theta) \\ \sin(\theta) & \cos(\theta) \end{bmatrix} \begin{bmatrix} x \\ y \end{bmatrix}$$

2. For **translation**, we use addition/subtraction:
   - x' = x - tₓ
   - y' = y - tᵧ
   
   Which we can write as:
   
   $$\begin{bmatrix} x' \\ y' \end{bmatrix} = \begin{bmatrix} x \\ y \end{bmatrix} - \begin{bmatrix} t_x \\ t_y \end{bmatrix}$$

3. When we have **both rotation and translation**, we need to combine these operations, if we want to transform a point P from frame B to frame A:
   
   $$^A P = {^A_B R} \cdot {^B P} + {^A T_B}$$
   
   Where:
   - $^A P$ is the point in frame A's coordinates
   - $^B P$ is the same point in frame B's coordinates
   - ${^A_B R}$ is the rotation matrix from frame B to frame A
   - ${^A T_B}$ is the translation vector from frame B's origin to frame A's origin

   In expanded form with our 2D rotation matrix:
   
   $$\begin{bmatrix} x_A \\ y_A \end{bmatrix} = \begin{bmatrix} \cos(\theta) & -\sin(\theta) \\ \sin(\theta) & \cos(\theta) \end{bmatrix} \begin{bmatrix} x_B \\ y_B \end{bmatrix} + \begin{bmatrix} T_x \\ T_y \end{bmatrix}$$

The problem is that we need both multiplication AND addition operations. We can't represent this transformation using just a single matrix multiplication with standard 2×2 matrices.

What we'd like is to express the entire transformation in one operation, like this:

$$^A P = \text{Some Matrix} \cdot {^B P}$$

This is where homogeneous coordinates come in - they allow us to represent both rotation and translation using a single matrix multiplication operation.

## 3. Homogeneous Coordinates: A Unified Approach

So far, we've seen that:
- Rotations are represented by 2×2 matrices
- Translations are represented by 2×1 vectors

This means we need to apply two separate operations to transform coordinates between frames that are both rotated and translated. Wouldn't it be nice if we could represent both transformations using a single matrix?

### 3.1 Introduction to Homogeneous Coordinates

Homogeneous coordinates add one extra dimension to our representation. In 2D space:

- A point (x, y) becomes (x, y, 1)
- A vector (x, y) becomes (x, y, 0)

This may seem strange at first, but it allows us to represent both rotation and translation in a single 3×3 matrix!

### 3.2 The Homogeneous Transformation Matrix

In homogeneous coordinates, a combined rotation and translation can be represented as:

$$\begin{bmatrix} x' \\ y' \\ 1 \end{bmatrix} = \begin{bmatrix} \cos(\theta) & -\sin(\theta) & t_x \\ \sin(\theta) & \cos(\theta) & t_y \\ 0 & 0 & 1 \end{bmatrix} \begin{bmatrix} x \\ y \\ 1 \end{bmatrix}$$

This 3×3 matrix is called the **homogeneous transformation matrix**. It can be broken down into blocks:

$$\begin{bmatrix} R & T \\ 0 & 1 \end{bmatrix}$$

Where:
- $R$ is the 2×2 rotation matrix
- $T$ is the 2×1 translation vector
- $0$ is a 1×2 row of zeros
- $1$ is a scalar

Let's implement this to transform our point:

In [10]:
# Define parameters for a combined transformation
point_coords = [1.5, 1.0]    # Point in world frame
tx, ty = 1.0, 0.5           # Translation vector
rotation = -30               # Rotation in degrees (negative for clockwise)
theta = radians(-rotation)   # Convert to positive radians for clockwise rotation

# Step 1: Create the rotation matrix
R = np.array([
    [cos(theta), -sin(theta)],
    [sin(theta), cos(theta)]
])

# Step 2: Create the homogeneous transformation matrix
H = np.array([
    [cos(theta), -sin(theta), tx],
    [sin(theta), cos(theta), ty],
    [0, 0, 1]
])

print("Homogeneous transformation matrix:")
print(H)

# Step 3: Convert point to homogeneous coordinates
point_homogeneous = np.array([point_coords[0], point_coords[1], 1])

# Step 4: Apply transformation
transformed_point_homogeneous = H @ point_homogeneous

# Step 5: Convert back to Cartesian coordinates (divide by last coordinate)
transformed_point = transformed_point_homogeneous[:2] / transformed_point_homogeneous[2]

print("\nPoint in world frame:", point_coords)
print("Point in homogeneous coordinates:", point_homogeneous)
print("Transformed point in homogeneous coordinates:", transformed_point_homogeneous)
print("Transformed point in Cartesian coordinates:", transformed_point)

# Let's verify this against our previous two-step calculation
# Step 1: Translation
x_translated = point_coords[0] - tx
y_translated = point_coords[1] - ty

# Step 2: Rotation
point_translated = np.array([x_translated, y_translated])
point_transformed_manual = R @ point_translated

print("\nVerification:")
print("Result using separate rotation and translation:", point_transformed_manual)
print("Result using homogeneous transformation matrix:", transformed_point)
print("Difference:", np.linalg.norm(point_transformed_manual - transformed_point))

Homogeneous transformation matrix:
[[ 0.8660254 -0.5        1.       ]
 [ 0.5        0.8660254  0.5      ]
 [ 0.         0.         1.       ]]

Point in world frame: [1.5, 1.0]
Point in homogeneous coordinates: [1.5 1.  1. ]
Transformed point in homogeneous coordinates: [1.79903811 2.1160254  1.        ]
Transformed point in Cartesian coordinates: [1.79903811 2.1160254 ]

Verification:
Result using separate rotation and translation: [0.1830127 0.6830127]
Result using homogeneous transformation matrix: [1.79903811 2.1160254 ]
Difference: 2.15987580880501


Wait a minute, the results don't match! This is because our homogeneous transformation matrix actually represents the opposite transformation sequence compared to what we did earlier.

### 3.3 Understanding the Order of Operations

When we use a homogeneous transformation matrix, the order of operations is implicit in the structure of the matrix:

$$\begin{bmatrix} R & T \\ 0 & 1 \end{bmatrix} \begin{bmatrix} x \\ y \\ 1 \end{bmatrix} = \begin{bmatrix} Rx + T \\ 1 \end{bmatrix}$$

This effectively applies the rotation first, then the translation. However, in our earlier manual approach, we applied translation first, then rotation.

Let's redefine our homogeneous transformation to match our earlier calculation:

In [11]:
# Create the correct homogeneous transformation matrix
# To match our earlier calculation (translate then rotate),
# we need to construct the matrix differently

# First create the translation matrix
T_matrix = np.array([
    [1, 0, -tx],  # Note the negative sign for translation to frame
    [0, 1, -ty],
    [0, 0, 1]
])

# Then create the rotation matrix in homogeneous form
R_matrix = np.array([
    [cos(theta), -sin(theta), 0],
    [sin(theta), cos(theta), 0],
    [0, 0, 1]
])

# Combined transformation: rotate after translate
H_correct = R_matrix @ T_matrix

print("Correct homogeneous transformation matrix:")
print(H_correct)

# Apply transformation
transformed_point_correct = H_correct @ point_homogeneous
transformed_point_cartesian = transformed_point_correct[:2] / transformed_point_correct[2]

print("\nVerification:")
print("Result using separate rotation and translation:", point_transformed_manual)
print("Result using correct homogeneous matrix:", transformed_point_cartesian)
print("Difference:", np.linalg.norm(point_transformed_manual - transformed_point_cartesian))

Correct homogeneous transformation matrix:
[[ 0.8660254 -0.5       -0.6160254]
 [ 0.5        0.8660254 -0.9330127]
 [ 0.         0.         1.       ]]

Verification:
Result using separate rotation and translation: [0.1830127 0.6830127]
Result using correct homogeneous matrix: [0.1830127 0.6830127]
Difference: 2.7755575615628914e-17


### 3.4 Advantages of Homogeneous Coordinates

Now that we understand how to correctly use homogeneous coordinates, let's explore their advantages:

1. **Unified representation**: A single matrix represents both rotation and translation
2. **Composing transformations**: Simply multiply the matrices together
3. **Inverting transformations**: The inverse of a transformation is simply the inverse of the matrix
4. **Extending to 3D**: The same approach works for 3D transformations (with 4×4 matrices)

Let's see how we can compose multiple transformations using homogeneous matrices:

In [12]:
# Define three transformations

# Transformation 1: Translate by (1, 0.5) and rotate 30° clockwise
tx1, ty1 = 1.0, 0.5
theta1 = radians(30)  # Convert to radians

# Transformation 2: Translate by (0, 1) and rotate 45° clockwise
tx2, ty2 = 0.0, 1.0
theta2 = radians(45)

# Transformation 3: Translate by (-0.5, 0) and rotate 15° clockwise
tx3, ty3 = -0.5, 0.0
theta3 = radians(15)

# Create individual homogeneous transformation matrices
# For transforming from world to frame, the matrices are:

# Translation matrices
T1 = np.array([
    [1, 0, -tx1],
    [0, 1, -ty1],
    [0, 0, 1]
])

T2 = np.array([
    [1, 0, -tx2],
    [0, 1, -ty2],
    [0, 0, 1]
])

T3 = np.array([
    [1, 0, -tx3],
    [0, 1, -ty3],
    [0, 0, 1]
])

# Rotation matrices
R1 = np.array([
    [cos(theta1), -sin(theta1), 0],
    [sin(theta1), cos(theta1), 0],
    [0, 0, 1]
])

R2 = np.array([
    [cos(theta2), -sin(theta2), 0],
    [sin(theta2), cos(theta2), 0],
    [0, 0, 1]
])

R3 = np.array([
    [cos(theta3), -sin(theta3), 0],
    [sin(theta3), cos(theta3), 0],
    [0, 0, 1]
])

# Combined transformations
H1 = R1 @ T1  # Transformation 1
H2 = R2 @ T2  # Transformation 2
H3 = R3 @ T3  # Transformation 3

# Composing transformations (equivalent to applying transformations in sequence)
# First transformation 1, then 2, then 3
H_combined = H3 @ H2 @ H1

print("Combined transformation matrix:")
print(H_combined)

# Apply the combined transformation to a point
point_coords = [1.5, 1.0]
point_homogeneous = np.array([point_coords[0], point_coords[1], 1])

# Transform the point
transformed_point = H_combined @ point_homogeneous
transformed_point_cartesian = transformed_point[:2] / transformed_point[2]

print("\nOriginal point:", point_coords)
print("Transformed point:", transformed_point_cartesian)

Combined transformation matrix:
[[ 2.14607521e-16 -1.00000000e+00  1.84898832e+00]
 [ 1.00000000e+00  2.07170437e-16 -1.37059048e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]

Original point: [1.5, 1.0]
Transformed point: [0.84898832 0.12940952]


This demonstrates how homogeneous coordinates allow us to easily compose multiple transformations by simply multiplying the corresponding matrices. This is a powerful concept that we'll use when we move to 3D transformations and camera models.

In the next section, we'll extend these concepts to 3D space.